In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2043,2024-06-13,Venezuela Superliga,19:45,Piratas de La Guaira,Centauros,1.39,2.84,151.5,1.86,1.84,0.0,0.00,0.00,bNG3EjLi,0.719424,0.352113,0.537634,0.543478,0.071537,191.760,93.019281,0.485082,1.8,1.643168,0.912871,105.60,2.316,1.067347,0.460858,22.0,142.390,41.028932,0.288145,0.0,0.000000,NaN,150.23,2.024,0.637519,0.314980,-49.0,80,83,1.32,1.81,109.610,155.656,0.484778,0.007644,NaN,0.75,0.150,2.600000,0.000000,0.0,0.000000,-3.02,-0.604,-3.046358,0.000000,0.0,0.000000
2044,2024-06-13,Polônia Liga De Basquete,15:00,Szczecin,Trefl Sopot,1.41,2.82,159.5,1.81,1.85,0.0,0.00,0.00,bk9Wjf4C,0.709220,0.354610,0.552486,0.540541,0.063830,163.494,24.826254,0.151848,1.8,1.643168,0.912871,176.40,1.968,0.317758,0.161462,20.0,142.250,23.648501,0.166246,1.8,1.643168,0.912871,149.60,1.776,0.329135,0.185324,-3.0,84,88,2.10,1.70,129.088,156.748,0.471405,0.015456,NaN,2.57,0.514,0.797665,0.732917,0.8,0.067083,-1.40,-0.280,-6.500000,0.596053,0.7,0.103947
2045,2024-06-13,Venezuela Superliga,13:00,Panteras,Guaiqueries,1.84,1.88,146.5,1.80,1.86,-1.5,1.97,2.05,fNLwdEUq,0.543478,0.531915,0.555556,0.537634,0.075393,208.596,67.430664,0.323260,1.8,1.643168,0.912871,192.28,2.776,0.861208,0.310234,-11.0,155.028,34.049730,0.219636,1.8,1.643168,0.912871,212.50,1.884,0.346742,0.184046,29.0,76,85,2.53,2.50,173.918,139.090,0.015207,0.023184,0.028144,-2.47,-0.494,-1.700405,0.000000,0.0,0.000000,-3.24,-0.648,-1.358025,0.000000,0.0,0.000000
2046,2024-06-13,Venezuela Superliga,22:00,Broncos,Spartans,1.36,2.97,167.5,1.85,1.81,0.0,0.00,0.00,EPyufhad,0.735294,0.336700,0.540541,0.552486,0.071994,131.724,30.114304,0.228617,3.0,0.000000,0.000000,183.00,1.476,0.206591,0.139967,23.0,182.746,59.767406,0.327052,1.2,1.643168,1.369306,140.65,2.194,0.669014,0.304929,18.0,100,97,1.83,1.45,0.000,140.638,0.525839,0.015456,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-3.55,-0.710,-2.774648,0.000000,0.0,0.000000
2047,2024-06-13,Paraguai Lnb,20:30,Felix Perez Cardozo,Colonias Gold,1.74,1.99,149.5,1.86,1.80,-2.5,2.01,2.40,0n5xPROG,0.574713,0.502513,0.537634,0.555556,0.077225,772.508,947.032721,1.225920,0.6,1.341641,2.236068,469.20,9.560,10.572575,1.105918,-18.0,230.448,144.765282,0.628191,1.8,1.643168,0.912871,268.11,2.806,1.562668,0.556902,12.0,68,81,6.90,3.31,677.742,0.000,0.094786,0.023184,0.125067,26.79,5.358,0.138111,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2048,2024-06-13,Canadá Cebl,21:00,Scarborough Shooting Stars,Ottawa Blackjacks,1.43,2.66,181.5,1.85,1.85,0.0,0.00,0.00,YwKYsu0D,0.699301,0.375940,0.540541,0.540541,0.075241,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.425301,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2049,2024-06-13,Canadá Cebl,23:30,Vancouver Bandits,Winnipeg Sea Bears,1.69,2.07,179.5,1.85,1.85,0.0,0.00,0.00,ns8G6OxE,0.591716,0.483092,0.540541,0.540541,0.074808,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.142926,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2050,2024-06-13,Colômbi

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
